In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)
x_train = sequence.pad_sequences(x_train, maxlen=80)
x_test = sequence.pad_sequences(x_test, maxlen=80)

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10831620027691362412
]


In [5]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[]

In [6]:
import tensorflow as tf

In [7]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.7
sess = tf.Session(config=config)
set_session(sess)

In [8]:
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=50,
          epochs=5,
          validation_data=(x_test, y_test))
model.evaluate(x_test, y_test, batch_size=50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 79s 3ms/step - loss: 0.4599 - accuracy: 0.7844 - val_loss: 0.3915 - val_accuracy: 0.8287
Epoch 2/5
25000/25000 [==============================] - 79s 3ms/step - loss: 0.2996 - accuracy: 0.8783 - val_loss: 0.3792 - val_accuracy: 0.8372
Epoch 3/5
25000/25000 [==============================] - 80s 3ms/step - loss: 0.2249 - accuracy: 0.9129 - val_loss: 0.4010 - val_accuracy: 0.8323
Epoch 4/5
25000/25000 [==============================] - 82s 3ms/step - loss: 0.1615 - accuracy: 0.9397 - val_loss: 0.4538 - val_accuracy: 0.8286
Epoch 5/5
25000/25000 [==============================] - 14s 563us/step


[0.5456346854567528, 0.823639988899231]

5 эпох полоучилось много. модель получилась переобученной.

Попробуем разные оптимайзеры

In [9]:
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=50,
          epochs=2,
          validation_data=(x_test, y_test))
model.evaluate(x_test, y_test, batch_size=50)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 74s 3ms/step - loss: 0.6930 - accuracy: 0.5055 - val_loss: 0.6930 - val_accuracy: 0.5036
Epoch 2/2
25000/25000 [==============================] - 13s 530us/step


[0.6928071086406707, 0.5289199948310852]

SGD плохо подходит.

In [11]:
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=50,
          epochs=2,
          validation_data=(x_test, y_test))
model.evaluate(x_test, y_test, batch_size=50)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 76s 3ms/step - loss: 0.4683 - accuracy: 0.7838 - val_loss: 0.3908 - val_accuracy: 0.8286
Epoch 2/2
25000/25000 [==============================] - 14s 565us/step


[0.4053425676524639, 0.8330399990081787]

In [12]:
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='Adamax',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=50,
          epochs=2,
          validation_data=(x_test, y_test))
model.evaluate(x_test, y_test, batch_size=50)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 79s 3ms/step - loss: 0.4724 - accuracy: 0.7746 - val_loss: 0.3612 - val_accuracy: 0.8392
Epoch 2/2
25000/25000 [==============================] - 15s 584us/step


[0.36223664590716365, 0.8406800031661987]

Самым лучшим оптимайзером оказался Adamax с итоговой точностью на валидационной выборке 0.84. Кол-во эпох было увеличино до 2.

In [13]:
from keras.layers.recurrent import GRU
from keras.layers import Activation

In [15]:
import numpy as np

In [16]:
with open("Data/alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])
    
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [21]:
model = Sequential()
model.add(
    GRU(128,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
model.fit(X, y, batch_size=128, epochs=2)

Epoch 1/2
158773/158773 [==============================] - 22s 137us/step - loss: 2.3124
Epoch 2/2
158773/158773 [==============================] - 21s 131us/step - loss: 1.9161


In [24]:
test_idx = np.random.randint(len(input_chars))
test_chars = input_chars[test_idx]

# для числа шагов предсказаний использование текущей тренируемой модели 
# конструирование one-hot encoding для тестирования input и добавление предсказания.
print("Генерация из посева: %s" % (test_chars))
print(test_chars, end="")
for i in range(25):

    # здесь one-hot encoding.
    X_test = np.zeros((1, SEQLEN, nb_chars))
    for j, ch in enumerate(test_chars):
        X_test[0, j, char2index[ch]] = 1

    # осуществление предсказания с помощью текущей модели.
    pred = model.predict(X_test, verbose=0)[0]
    y_pred = index2char[np.argmax(pred)]

    # вывод предсказания добавленного к тестовому примеру 
    print(y_pred, end="")

    # инкрементация тестового примера содержащего предсказание
    test_chars = test_chars[1:] + y_pred

Генерация из посева:  forth in 
 forth in a said the cane the mack 

Попробуем еще увеличить число эпох.

In [25]:
model = Sequential()
model.add(
    GRU(128,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
model.fit(X, y, batch_size=128, epochs=4)

Epoch 1/4
158773/158773 [==============================] - 23s 145us/step - loss: 2.3062
Epoch 2/4
158773/158773 [==============================] - 22s 140us/step - loss: 1.9111
Epoch 3/4
158773/158773 [==============================] - 22s 140us/step - loss: 1.7568
Epoch 4/4
158773/158773 [==============================] - 22s 140us/step - loss: 1.6556


In [27]:
test_idx = np.random.randint(len(input_chars))
test_chars = input_chars[test_idx]

# для числа шагов предсказаний использование текущей тренируемой модели 
# конструирование one-hot encoding для тестирования input и добавление предсказания.
print("Генерация из посева: %s" % (test_chars))
print(test_chars, end="")
for i in range(25):

    # здесь one-hot encoding.
    X_test = np.zeros((1, SEQLEN, nb_chars))
    for j, ch in enumerate(test_chars):
        X_test[0, j, char2index[ch]] = 1

    # осуществление предсказания с помощью текущей модели.
    pred = model.predict(X_test, verbose=0)[0]
    y_pred = index2char[np.argmax(pred)]

    # вывод предсказания добавленного к тестовому примеру 
    print(y_pred, end="")

    # инкрементация тестового примера содержащего предсказание
    test_chars = test_chars[1:] + y_pred

Генерация из посева: rown so la
rown so latter all she said the mor

В этот раз фраза получилась более оосмысленной.

In [28]:
test_idx = np.random.randint(len(input_chars))
test_chars = input_chars[test_idx]

# для числа шагов предсказаний использование текущей тренируемой модели 
# конструирование one-hot encoding для тестирования input и добавление предсказания.
print("Генерация из посева: %s" % (test_chars))
print(test_chars, end="")
for i in range(30):

    # здесь one-hot encoding.
    X_test = np.zeros((1, SEQLEN, nb_chars))
    for j, ch in enumerate(test_chars):
        X_test[0, j, char2index[ch]] = 1

    # осуществление предсказания с помощью текущей модели.
    pred = model.predict(X_test, verbose=0)[0]
    y_pred = index2char[np.argmax(pred)]

    # вывод предсказания добавленного к тестовому примеру 
    print(y_pred, end="")

    # инкрементация тестового примера содержащего предсказание
    test_chars = test_chars[1:] + y_pred

Генерация из посева: g at the h
g at the hatter some of the sore of the 

В итоге для более-менее осмысленных фраз пришлось увеличить кол-во эпох обучения до 4.